# Word Tokenization

In [1]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
sentences = ['I love my cat','I love my dog','I love my dog!', 'I have too many dogs','I','anticipate']

We saw "Teddy Roosevelt was a great President"

We saw "Teddy bears were at a discount"

In [3]:
tokenizer = Tokenizer(num_words = 10, oov_token='<UKW>')

In [4]:
tokenizer.fit_on_texts(sentences)

In [5]:
print(tokenizer.word_index)

{'<UKW>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'cat': 6, 'have': 7, 'too': 8, 'many': 9, 'dogs': 10, 'anticipate': 11}


In [6]:
sequences = tokenizer.texts_to_sequences(sentences)

In [7]:
sequences

[[2, 3, 4, 6], [2, 3, 4, 5], [2, 3, 4, 5], [2, 7, 8, 9, 1], [2], [1]]

In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
sequences = pad_sequences(sequences, padding='post', truncating='post')

In [10]:
sequences

array([[2, 3, 4, 6, 0],
       [2, 3, 4, 5, 0],
       [2, 3, 4, 5, 0],
       [2, 7, 8, 9, 1],
       [2, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)

# Sentiment Analyzer

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.head()

In [ ]:
data['sentiment'].unique()

In [ ]:
from nltk.corpus import stopwords

In [ ]:
english_stops = set(stopwords.words('english'))

In [ ]:
#print(english_stops)

In [ ]:
X_data = data['review']
y_data = data['sentiment']

In [ ]:
X_data = X_data.replace({'<.*?>': ''}, regex = True) # remove html tag

In [ ]:
X_data = X_data.replace({'[^A-Za-z]': ' '}, regex = True) # remove non alphabetical characters

In [ ]:
X_data = X_data.apply(lambda review: [w for w in review.split() if w not in english_stops]) # Stopwords

In [ ]:
X_data = X_data.apply(lambda review: [w.lower() for w in review]) # Convert to lowercase

In [ ]:
y_data = y_data.replace('positive', 1)
y_data = y_data.replace('negative', 0)

In [ ]:
#X_data[0:5]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, 
                                                    y_data, 
                                                    test_size = 0.2, 
                                                    random_state=12345)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
def get_max_length():
    review_length = []
    for review in X_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [ ]:
# to encode text to int
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
token = Tokenizer(lower=False)

In [ ]:
token.fit_on_texts(X_train)

In [ ]:
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [ ]:
max_length = get_max_length()

In [ ]:
max_length

In [ ]:
X_train = pad_sequences(X_train, 
                        maxlen=max_length, 
                        padding='post', 
                        truncating='post')

In [ ]:
X_test = pad_sequences(X_test, 
                       maxlen=max_length, 
                       padding='post', 
                       truncating='post')

In [ ]:
total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', X_train, '\n')
print('Encoded X Test\n', X_test, '\n')
print('Maximum review length: ', max_length)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
model = Sequential()

In [ ]:
model.add(Embedding(total_words, 
                    32, 
                    input_length = max_length))

In [ ]:
model.add(LSTM(256, 
               return_sequences=True))
model.add(LSTM(256,return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train, 
          y_train,
          validation_data=(X_test,y_test), 
          epochs = 5)

In [ ]:
y_test_pred = model.predict(X_test)

In [ ]:
y_test_pred = y_test_pred >= 0.5

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
confusion_matrix(y_pred = y_test_pred, y_true = y_test)

In [ ]:
accuracy_score(y_pred = y_test_pred, y_true = y_test)